In [1]:
import numpy as np
import awkward as ak
from tqdm import tqdm
import uproot


data_direc = f"/vols/cms/emc21/fccStudy/data"



In [2]:
import glob

def getRunLoc(directory, prefix=False):
    # Get a directory for the run: There might have been previous
    # runs in the same directory, if so we don't want to overwite
    # -> automatically +1 to end of the run name
    direcs = glob.glob(f"{directory}/*/")
    runs = sorted(
        [int(direc.split("/")[-2].split("run")[1]) for direc in direcs]
    )
    if len(runs) == 0:  # if no runs yet
        run_name = f"run1"
    else:  # if there are runs, add 1 to end for new run name
        n = runs[-1] + 1
        run_name = f"run{n}"

    if prefix:
        run_name = f"{prefix}_{run_name}"

    run_loc = f"{directory}/{run_name}"
    print(f"======= Run directory: =======")
    print(run_loc)
    print(f"==============================")
    # Make the run_loc
    # os.makedirs(run_loc, exist_ok=True)

    return run_loc


base_run_dir = "runs/e240_run"
run_loc = getRunLoc(base_run_dir)

run_loc

======= Run directory: =======
runs/e240_run/run1


'runs/e240_run/run1'

In [2]:
def flattenFields(evs):
    for field in ak.fields(evs):
        if "var" in str(ak.type(evs[field])):
            evs[field] = ak.flatten(evs[field])
    return evs

In [3]:
def getWeight(evs, xs, lumi):
    n_samples = len(evs)
    weight = xs * lumi / n_samples

    return weight


def getData(samples, cuts=None):

    events = []
    # First load in the signal samples
    for sig_point, sig_dict in samples['signal'].items():
        print(f"Loading signal point: {sig_point}")
        files = sig_dict["files"]
        files = [f"{samples['directory']}/{f}" for f in files]

        for file in tqdm(files):
            print(f"Loading file: {file.split('/')[-1]}")
            with uproot.open(file, num_workers=16) as f:
                tree = f["events"]
                branches_to_load = list(tree.keys())
                branches_to_load.remove("n_seljets")
                evs = tree.arrays(branches_to_load, library="ak")

                weight = getWeight(evs, sig_dict["xs"], samples["Luminosity"])
                evs["weight"] = ak.ones_like(evs.Zcand_m) * weight

                if cuts is not None:
                    evs = cuts(evs)

                # I want to add some stuff here as well to make the data more useful
                class_num = ak.ones_like(evs.Zcand_m)
                # Get the BP number
                id_num = int(sig_point.split("BP")[1])
                evs["id_num"] = ak.ones_like(evs.Zcand_m) * id_num
                # Also get the masses
                mH = sig_dict["masses"][0]
                mA = sig_dict["masses"][1]
                evs["mH"] = ak.ones_like(evs.Zcand_m) * mH
                evs["mA"] = ak.ones_like(evs.Zcand_m) * mA

                process = sig_point
                specific_proc = file.split("_")[-1].split(".root")[0]

                evs["process"] = [process] * len(evs.Zcand_m)
                evs["specific_proc"] = [specific_proc] * len(evs.Zcand_m)
                evs["class"] = class_num

                # flatten all fields
                evs = flattenFields(evs)
                # Convert all to float32
                evs = ak.values_astype(evs, "float32")

                events.append(evs)

    # Now load in the background samples
    for proc, proc_dict in samples['backgrounds'].items():
        print(f"Loading background process: {proc}")
        files = proc_dict["files"]
        files = [f"{samples['directory']}/{f}" for f in files]

        for file in tqdm(files):
            print(f"Loading file: {file.split('/')[-1]}")
            with uproot.open(file, num_workers=16) as f:
                tree = f["events"]
                branches_to_load = list(tree.keys())
                branches_to_load.remove("n_seljets")
                evs = tree.arrays(branches_to_load, library="ak")

                weight = getWeight(evs, sig_dict["xs"], samples["Luminosity"])
                evs["weight"] = ak.ones_like(evs.Zcand_m) * weight

                if cuts is not None:
                    evs = cuts(evs)

                # I want to add some stuff here as well to make the data more useful
                class_num = ak.zeros_like(evs.Zcand_m)

                evs["mH"] = [-1] * len(evs)
                evs["mA"] = [-1] * len(evs)
                evs["id_num"] = [-1] * len(evs)

                evs["process"] = [proc] * len(evs.Zcand_m)
                evs["specific_proc"] = [proc] * len(evs.Zcand_m)
                evs["class"] = class_num

                # flatten all fields
                evs = flattenFields(evs)
                # Convert all to float32
                evs = ak.values_astype(evs, "float32")

                events.append(evs)
    
    events = ak.concatenate(events)
    return events

In [4]:
samples = {
    "data_directory" : "data",
    "backgrounds" : {
    "wzp6_ee_mumu_ecm240": {
        "files" : ["wzp6_ee_mumu_ecm240.root"],
        "xs": 5.288
    },
    "wzp6_ee_tautau_ecm240": {
        "files" : ["wzp6_ee_tautau_ecm240.root"],
        "xs": 4.668
    }
    },
    "signal" : {
        "BP1" : {
            "files" : ["e240_bp1_h2h2ll.root", "e240_bp1_h2h2llvv.root"],
            "masses" : [80, 150],
            "xs": 0.0069
        },
        "BP2" : {
            "files" : ["e240_bp2_h2h2ll.root", "e240_bp2_h2h2llvv.root"],
            "masses" : [80, 160],
            "xs": 0.005895
        },
    },
    "Luminosity" : 500,
    "directory" : data_direc,
    "test_size" : 0.25 # e.g. 0.2 means 20% of data used for test set
    }

In [5]:
def applyCuts(evs):
    print(f"Applying Cuts")
    mask = (
        (np.abs(evs.Zcand_pz) < 70)
        & (evs.jet1_e < 0)
        & (evs.n_photons == 0)
        & (evs.MET_pt > 5)
        & (evs.lep1_pt < 80)
        & (evs.lep2_pt < 60)
        & (evs.Zcand_povere > 0.1)
    )
    mask = ak.flatten(mask)
    sum_before = ak.sum(evs.weight)
    sum_after = ak.sum(evs[mask].weight)
    print(
        f"Sum before: {sum_before}, Sum after: {sum_after}, Fraction: {sum_after/sum_before}"
    )
    return evs[mask]


data = getData(samples, applyCuts)

Loading signal point: BP1


  0%|          | 0/2 [00:00<?, ?it/s]

Loading file: e240_bp1_h2h2ll.root


Applying Cuts
Sum before: 3.4393792152404785, Sum after: 3.1485276222229004, Fraction: 0.9154348373413086


 50%|█████     | 1/2 [00:04<00:04,  4.26s/it]

...

100%|██████████| 1/1 [00:04<00:00,  4.26s/it]


Loading background process: wzp6_ee_tautau_ecm240


  0%|          | 0/1 [00:00<?, ?it/s]

Loading file: wzp6_ee_tautau_ecm240.root
Applying Cuts
Sum before: 2.974040985107422, Sum after: 1.8348617553710938, Fraction: 0.6169591546058655


100%|██████████| 1/1 [00:18<00:00, 18.34s/it]


In [10]:

branches = ['Zcand_m',
 'Zcand_pt',
 'Zcand_pz',
 'Zcand_p',
 'Zcand_povere',
 'Zcand_e',
 'Zcand_costheta',
 'Zcand_recoil_m',
 'photon1_pt',
 'photon1_eta',
 'photon1_e',
 'lep1_pt',
 'lep1_eta',
 'lep1_e',
 'lep1_charge',
 'lep2_pt',
 'lep2_eta',
 'lep2_e',
 'lep2_charge',
 'lep_chargeprod',
 'jet1_pt',
 'jet1_eta',
 'jet1_e',
 'jet2_pt',
 'jet2_eta',
 'jet2_e',
 'cosDphiLep',
 'cosThetaStar',
 'cosThetaR',
 'n_jets',
 'MET_e',
 'MET_pt',
 'MET_eta',
 'MET_phi',
 'n_photons',
 'n_muons',
 'n_electrons']


for b in branches:
    print(f"{b: <15}, mean = {np.mean(data[b]**2):.2f}")

Zcand_m        , mean = 2412.65
Zcand_pt       , mean = 578.06
Zcand_pz       , mean = 715.23
Zcand_p        , mean = 1293.54
Zcand_povere   , mean = 0.50
Zcand_e        , mean = 3706.33
Zcand_costheta , mean = 0.45
Zcand_recoil_m , mean = 34162.81
photon1_pt     , mean = 1.00
photon1_eta    , mean = 24.37
photon1_e      , mean = 1.00
lep1_pt        , mean = 999.55
lep1_eta       , mean = 0.70
lep1_e         , mean = 1757.68
lep1_charge    , mean = 1.00
lep2_pt        , mean = 232.62
lep2_eta       , mean = 1.04
lep2_e         , mean = 460.34
lep2_charge    , mean = 1.00
lep_chargeprod , mean = 1.00
jet1_pt        , mean = 1.00
jet1_eta       , mean = 24.37
jet1_e         , mean = 1.00
jet2_pt        , mean = 1.00
jet2_eta       , mean = 24.37
jet2_e         , mean = 1.00
cosDphiLep     , mean = 0.84
cosThetaStar   , mean = 0.33
cosThetaR      , mean = 0.33
n_jets         , mean = 2.97
MET_e          , mean = 1304.22
MET_pt         , mean = 579.92
MET_eta        , mean = 1.50
MET_phi  

In [20]:
data.photon1_pt

<Array [-1, -1, -1, -1, -1, ..., -1, -1, -1, -1, -1] type='1801498 * float32'>

In [15]:
ak.any(data.jet1_pt != -1)

False

In [25]:
procs = np.unique(data.process)
for proc in procs:
    weights = data[data.process == proc].weight
    print(weights)
    print(f"Process: {proc}, Number of events: {np.sum(weights)}")

[1.1e-05, 1.1e-05, 1.1e-05, 1.1e-05, ..., 2.68e-05, 2.68e-05, 2.68e-05]
Process: BP1, Number of events: 6.24392557144165
[1.65e-05, 1.65e-05, 1.65e-05, 1.65e-05, ..., 2.27e-05, 2.27e-05, 2.27e-05]
Process: BP2, Number of events: 5.597738265991211
[8.42e-06, 8.42e-06, 8.42e-06, 8.42e-06, ..., 8.42e-06, 8.42e-06, 8.42e-06]
Process: wzp6_ee_mumu_ecm240, Number of events: 0.77073734998703
[1.82e-06, 1.82e-06, 1.82e-06, 1.82e-06, ..., 1.82e-06, 1.82e-06, 1.82e-06]
Process: wzp6_ee_tautau_ecm240, Number of events: 1.8348617553710938


In [27]:
from sklearn.model_selection import train_test_split
def consistentTrainTestSplit(
    events, test_size=0.2, random_state=42, stratify_var="process"
):

    print("Doing train test split")
    unique_strat_vars = list(np.unique(events[stratify_var]))
    print(unique_strat_vars)

    # Split using indexes as it is much faster
    indexes = np.arange(len(events))

    train_data_idxs, test_data_idxs = train_test_split(
        indexes,
        test_size=test_size,
        random_state=random_state,
        stratify=events[stratify_var],
    )

    train_data = events[train_data_idxs]
    test_data = events[test_data_idxs]

    train_data["weight_scaled"] = train_data["weight"] / (1 - test_size)
    test_data["weight_scaled"] = test_data["weight"] / test_size

    return train_data, test_data


# Now split into train and test
train_data, test_data = consistentTrainTestSplit(data, 
                                                 test_size=samples['test_size'])

Doing train test split
['BP1', 'BP2', 'wzp6_ee_mumu_ecm240', 'wzp6_ee_tautau_ecm240']


In [28]:
def normaliseWeights(events):
    # Normalise so the signal process sum to the same weight
    # Start by making all signal samples to sum to 1
    sig_dat = events[events["class"] == 1]
    signal_procs = np.unique(sig_dat["process"])

    # Want sum of weights of each signal to equal 1, then I will reweight
    # both such that the average weight = 0.001
    for proc in signal_procs:
        print(proc)
        proc_data = events[events["process"] == proc]
        sum_weight = np.sum(proc_data["weight"])

        events["weight"] = ak.where(
            events["process"] == proc,
            events["weight"] / sum_weight,
            events["weight"],
        )

    # now reweight so that the average weight of signal = 0.001
    mean_sig = np.mean(events[events["class"] == 1]["weight"])
    ratio = 0.001 / mean_sig
    events["weight"] = events["weight"] * ratio

    # Now get the sum of signal and reweight background to that
    sum_sig = np.sum(events[events["class"] == 1]["weight"])
    sum_bkg = np.sum(events[events["class"] == 0]["weight"])
    events["weight"] = ak.where(
        events["class"] == 0,
        events["weight"] * (sum_sig / sum_bkg),
        events["weight"],
    )

    # Now check that the sum of signal and background are the same
    sig = events[events["class"] == 1]
    bkg = events[events["class"] == 0]
    sum_sig = np.floor(np.sum(sig['weight']))
    sum_bkg = np.floor(np.sum(bkg['weight']))
    print(f"Sum sig = {sum_sig}, sum bkg = {sum_bkg}")

    if abs(sum_sig - sum_bkg) > 2:
        print("Sum sig does not equal sum background!")

    return events

# sum(backgrounds) = sum(signal)
train_data = normaliseWeights(train_data)
test_data = normaliseWeights(test_data)


BP1
BP2
Sum sig = 522.0, sum bkg = 522.0
BP1
BP2
Sum sig = 174.0, sum bkg = 174.0


In [34]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

def convertToNumpy(events: ak.Array, branches: list) -> np.array:
    # if only 1 branch then don't do the view stuff below, as that returns
    # garbage
    if len(branches) == 1:
        return ak.to_numpy(events[branches[0]])

    numpy_data = (
        ak.to_numpy(events[branches]).view("<f4").reshape(-1, len(branches))
    )
    return numpy_data

def applyScaler(evs, scaler, branches):
    numpy_data = convertToNumpy(evs, branches)
    trans_numpy_data = scaler.transform(numpy_data)
    for i, br in enumerate(branches):
        evs[br] = trans_numpy_data[:, i]

    return evs


def scaleFeatures(train_data, test_data, branches, run_loc="."):
    print(f"Scaling features")
    scaler = StandardScaler()
    numpy_data = convertToNumpy(train_data, branches)
    scaler.fit(numpy_data)
    del numpy_data

    train_data = applyScaler(train_data, scaler, branches)
    test_data = applyScaler(test_data, scaler, branches)

    # Also scale the masses using the minmax scaler
    # First select just the signal
    sig_data = train_data[train_data["class"] == 1]
    mass_branches = ["mH", "mA"]
    mass_scaler = MinMaxScaler()
    numpy_data = convertToNumpy(sig_data, mass_branches)
    mass_scaler.fit(numpy_data)

    del numpy_data

    train_data = applyScaler(train_data, mass_scaler, mass_branches)
    test_data = applyScaler(test_data, mass_scaler, mass_branches)

    # Now save them both
    pickle.dump(scaler, open(f"{run_loc}/scaler.pkl", "wb"))
    pickle.dump(mass_scaler, open(f"{run_loc}/mass_scaler.pkl", "wb"))

    return train_data, test_data, scaler, mass_scaler


branches = ['Zcand_m',
 'Zcand_pt',
 'Zcand_pz',
 'Zcand_p',
 'Zcand_povere',
 'Zcand_e',
 'Zcand_costheta',
 'Zcand_recoil_m',
 'photon1_pt',
 'photon1_eta',
 'photon1_e',
 'lep1_pt',
 'lep1_eta',
 'lep1_e',
 'lep1_charge',
 'lep2_pt',
 'lep2_eta',
 'lep2_e',
 'lep2_charge',
 'lep_chargeprod',
 'jet1_pt',
 'jet1_eta',
 'jet1_e',
 'jet2_pt',
 'jet2_eta',
 'jet2_e',
 'cosDphiLep',
 'cosThetaStar',
 'cosThetaR',
 'n_jets',
 'MET_e',
 'MET_pt',
 'MET_eta',
 'MET_phi',
 'n_photons',
 'n_muons',
 'n_electrons']

run_loc = "test"

# Now scale the features so that intput features have mean 0 and std 1
# and that the masses are scaled to be between 0 and 1
train_data, test_data, feat_scaler, mass_scaler = scaleFeatures(train_data, 
                                                                test_data, 
                                                                branches,
                                                                run_loc)

Scaling features


In [36]:
for branch in branches:
    bdata = test_data[branch]
    print(f"Branch: {branch}, mean: {np.mean(bdata)}, std: {np.std(bdata)}")

Branch: Zcand_m, mean: -0.0001683638099164325, std: 1.0015350967692098
Branch: Zcand_pt, mean: 0.0029507338719469887, std: 1.0007397873190778
Branch: Zcand_pz, mean: 0.0017798503417291145, std: 0.9995208866371187
Branch: Zcand_p, mean: 0.0014613517869223563, std: 0.9994377833407256
Branch: Zcand_povere, mean: 0.0018473654224777964, std: 1.0003683340286504
Branch: Zcand_e, mean: 0.0008332639467110741, std: 1.00061710744284
Branch: Zcand_costheta, mean: 0.0014219622753608105, std: 0.9994216638513411
Branch: Zcand_recoil_m, mean: -0.0008649232193658063, std: 1.000430597637535
Branch: photon1_pt, mean: 0.0, std: 0.0
Branch: photon1_eta, mean: 0.0, std: 0.0
Branch: photon1_e, mean: 0.0, std: 0.0
Branch: lep1_pt, mean: -4.521254159131713e-05, std: 1.0006674143363432
Branch: lep1_eta, mean: 0.0033283518617298084, std: 1.0024313125018913
Branch: lep1_e, mean: 0.0018345988260650846, std: 1.0011671128722135
Branch: lep1_charge, mean: 0.0012667559206303775, std: 0.9999991976643969
Branch: lep2_pt

In [38]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(
        self,
        data,
        train_branches,
        feat_scaler,
        mass_scaler,
        mass_branches=["mH", "mA"],
    ):
        print("Initialising CustomDataset")
        self.train_branches = train_branches
        self.data = data

        # Get the features and masses
        self.features = convertToNumpy(data, train_branches)
        self.masses = convertToNumpy(data, mass_branches)
        labs = convertToNumpy(data, ["class"])
        self.labels = np.reshape(labs, (len(labs), 1))
        weights = convertToNumpy(data, ["weight"])
        self.weight = np.reshape(weights, (len(weights), 1))

        if not "weight_scaled" in ak.fields(data):
            data["weight_scaled"] = data["weight"]

        weights_scaled = convertToNumpy(data, ["weight_scaled"])
        self.weight_scaled = np.reshape(weights_scaled, (len(weights_scaled), 1))

        # Now find the unique masses
        # Pick only the signal masses, not the default ones for the backgrounds
        sig_masses = self.masses[labs == 1]
        self.unique_masses = np.unique(sig_masses, axis=0)

        # Also save the scalers so that we can convert between the real data
        # and the converted data
        self.feat_scaler = feat_scaler
        self.mass_scaler = mass_scaler

    def shuffleMasses(self):
        # Want to find unique groupings of the mH,mA,mHch
        # Pick masses randomly from unique_masses
        choices = np.random.choice(
            np.arange(len(self.unique_masses)), len(self.labels)
        )

        shuffled_masses = self.unique_masses[choices]

        # Now mask so that the original signal masses are not changed
        sig_mask = self.labels == 1
        new_masses = np.where(sig_mask, self.masses, shuffled_masses)

        self.masses = new_masses

    def setAllMasses(self, masses_to_set):
        # Set all backgrounds to the same mass
        # e.g. masses_to_set = [80, 100, 160]
        # Expand out to same size of data array
        specific_masses = np.array([masses_to_set] * len(self.labels))

        # Now only apply the background samples
        sig_mask = self.labels == 1
        new_masses = np.where(sig_mask, self.masses, specific_masses)

        self.masses = new_masses

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        mass = self.masses[idx]
        w = self.weight[idx]
        wc = self.weight_scaled[idx]
        # self.features = convertToNumpy(self.data[idx], self.train_branches)
        # self.masses = convertToNumpy(self.data[idx], self.mass_branches)
        # self.labels = convertToNumpy(self.data[idx], ['class'])
        # self.weight = convertToNumpy(self.data[idx], ['weight'])

        return x, y, mass, w, wc

    def toOriginalArray(self):
        # Want to change back the self.data features and masses
        features_nump = convertToNumpy(self.data, self.train_branches)
        features_nump = self.feat_scaler.inverse_transform(features_nump)

        mass_nump = convertToNumpy(self.data, self.mass_branches)
        mass_nump = self.mass_scaler.inverse_transform(mass_nump)

        for i, field in enumerate(self.train_branches):
            self.data[field] = features_nump[:, i]

        for i, field in enumerate(self.mass_branches):
            self.data[field] = mass_nump[:, i]

        return self.data

# # Now put these both into helper classes for training 
train_dataset = CustomDataset(train_data, branches, feat_scaler, mass_scaler)
test_dataset = CustomDataset(test_data, branches, feat_scaler, mass_scaler)
train_dataset.shuffleMasses()
test_dataset.shuffleMasses()


Initialising CustomDataset


Initialising CustomDataset


: 